## Map Mutations to 3D Structures in the Protein Data Bank

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.datasets import g2sDataset, pdbjMineDataset, pdbToUniProt
from ipywidgets import interact, IntSlider
import py3Dmol
import pandas as pd

In [2]:
# Initialize Spark
spark = SparkSession.builder.master("local[4]").appName("2-MapTo3DStructures").getOrCreate()
#spark.conf.set("spark.driver.maxResultSize", "2g")
spark.sql("SET").toPandas()

,key,value
0,spark.app.id,local-1549605868862
1,spark.app.name,2-MapTo3DStructures
2,spark.driver.host,192.168.1.106
3,spark.driver.memory,20G
4,spark.driver.port,54809
5,spark.executor.id,driver
6,spark.master,local[4]
7,spark.rdd.compress,True
8,spark.serializer.objectStreamReset,100
9,spark.submit.deployMode,client


#### Input parameters

In [3]:
distance_cutoff = 8 # distance cutoff for visualizing interactions
input_file_name = 'mutations.csv' # mutations in standard format (e.g., chr5:g.149440497C>T)
output_file_name = 'mutations3d.csv' # mutations mapped to 3D protein structures

## Read 'mutations.csv' file created in the previous step

In [4]:
df = pd.read_csv(input_file_name)

## Create a list of the variants

In [5]:
var_ids = df['var_id'].tolist()
var_ids

['chr6:g.52619766C>T', 'chr9:g.133738358A>T', 'chr11:g.5246945G>T']

## Map the mutations from genome locations to 3D PDB positions¶
Here we use [g2sDataset](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/datasets/g2sDataset.py) to retrieve genome to PDB mapping information using the [G2S](https://g2s.genomenexus.org/) (Genome to Structure) web services.

Note, this step may take several minutes to complete.

In [6]:
#pdb_map = g2sDataset.get_position_dataset(var_ids, ref_genome='hgvs-grch37').toPandas()
pdb_map = g2sDataset.get_full_dataset(var_ids, ref_genome='hgvs-grch37').toPandas()

### Reduce redundant data to speed up this demo.
For you own data you may want to comment out this step to make sure you are getting all PDB mappings

In [7]:
pdb_map = pdb_map.drop_duplicates(subset=['structureId','pdbPosition','pdbAminoAcid','variationId'])

In [8]:
pd.set_option('display.max_columns', None)  # show all columns
# pd.set_option('display.max_colwidth', -1)  # don't truncate strings (show entire sequence)
pdb_map.head()

,alignmentId,bitscore,chainId,evalue,identity,identityPositive,midlineAlign,pdbAlign,pdbFrom,pdbId,pdbNo,pdbSeg,pdbTo,refGenome,residueMapping,segStart,seqAlign,seqFrom,seqId,seqTo,updateDate,variationId,structureId,pdbPosition,pdbAminoAcid
0,15184664,446.432,H,1.83875e-159,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2wju,2wju_H_1,1,221,hgvs-grch37,"[(G, 83, G, 83)]",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,2WJU,83,G
8,15184672,446.432,H,1.83875e-159,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2vct,2vct_H_1,1,221,hgvs-grch37,"[(G, 83, G, 83)]",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,2VCT,83,G
16,15184682,431.409,B,1.96091e-153,213.0,216.0,AEKPKLHY N RGRMES RWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYFNARGRMESTRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,1ags,1ags_B_1,1,221,hgvs-grch37,"[(R, 82, G, 83)]",1,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,1AGS,82,R
18,15184686,425.631,B,3.81593e-151,210.0,214.0,AEKPKLHY N RGRMES RWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYFNARGRMESTRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,1pl1,1pl1_B_1,1,221,hgvs-grch37,"[(G, 83, G, 83)]",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,1PL1,83,G
20,15184688,425.631,B,3.81593e-151,210.0,214.0,AEKPKLHY N RGRMES RWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYFNARGRMESTRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,1pkw,1pkw_B_1,1,221,hgvs-grch37,"[(G, 83, G, 83)]",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,1PKW,83,G


### Filter by sequence identity to PDB sequence

In [9]:
pdb_map['seqIdentity'] = pdb_map.identity/(pdb_map.seqTo - pdb_map.seqFrom + 1) * 100
pdb_map = pdb_map[pdb_map.seqIdentity >= 98]
pdb_map

,alignmentId,bitscore,chainId,evalue,identity,identityPositive,midlineAlign,pdbAlign,pdbFrom,pdbId,pdbNo,pdbSeg,pdbTo,refGenome,residueMapping,segStart,seqAlign,seqFrom,seqId,seqTo,updateDate,variationId,structureId,pdbPosition,pdbAminoAcid,seqIdentity
0,15184664,446.432,H,1.83875e-159,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2wju,2wju_H_1,1,221,hgvs-grch37,"[(G, 83, G, 83)]",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,2WJU,83,G,99.547511
8,15184672,446.432,H,1.83875e-159,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2vct,2vct_H_1,1,221,hgvs-grch37,"[(G, 83, G, 83)]",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,2VCT,83,G,99.547511
41,32437406,432.180,C,7.6822e-158,215.0,215.0,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,1,4acs,4acs_C_1,1,218,hgvs-grch37,"[(G, 83, G, 83)]",4,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,4,434735,221,2018-05-03,chr6:g.52619766C>T,4ACS,83,G,98.623853
100,17063512,993.416,A,0,465.0,466.0,ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...,ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...,1,2fo0,2fo0_A_1,1,466,hgvs-grch37,"[(Y, 272, Y, 253)]",65,ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...,46,488093,511,2017-10-25,chr9:g.133738358A>T,2FO0,272,Y,99.785408
101,17063513,961.444,A,0,450.0,451.0,DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...,DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...,1,1opl,1opl_A_1,1,451,hgvs-grch37,"[(Y, 272, Y, 253)]",81,DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...,62,488093,512,2017-10-25,chr9:g.133738358A>T,1OPL,272,Y,99.778271
102,17063514,956.051,A,0,446.0,449.0,NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQG...,NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQG...,1,1opk,1opk_A_1,1,449,hgvs-grch37,"[(Y, 272, Y, 253)]",83,NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQG...,64,488093,512,2017-10-25,chr9:g.133738358A>T,1OPK,272,Y,99.331849
103,17063515,671.003,A,0,315.0,316.0,NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQG...,NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQG...,1,5mo4,5mo4_A_1,1,319,hgvs-grch37,"[(Y, 272, Y, 253)]",83,NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQG...,64,488093,382,2017-10-25,chr9:g.133738358A>T,5MO4,272,Y,98.746082
104,17063516,610.912,A,0,285.0,286.0,G+SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKT...,GMSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKT...,1,2f4j,2f4j_A_1,1,287,hgvs-grch37,"[(Y, 253, Y, 253)]",227,GVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKT...,227,488093,513,2017-10-25,chr9:g.133738358A>T,2F4J,253,Y,99.303136
105,17063517,608.216,B,0,283.0,284.0,SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLK...,SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLK...,5,1opj,1opj_B_1,1,288,hgvs-grch37,"[(Y, 272, Y, 253)]",244,SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLK...,229,488093,512,2017-10-25,chr9:g.133738358A>T,1OPJ,272,Y,99.647887
106,17063520,607.446,B,0,282.0,284.0,SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLK...,SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLK...,2,3ik3,3ik3_B_1,1,286,hgvs-grch37,"[(Y, 253, Y, 253)]",228,SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLK...,229,488093,513,2017-10-25,chr9:g.133738358A>T,3IK3,253,Y,98.947368


In [10]:
# You can also get the mapping for a specific PDB chain, e.g.,

#pdb_map = g2sDataset.get_position_dataset(var_ids, '4ACS', 'B', ref_genome='hgvs-grch37').toPandas()

## Filter PDB Chains

Here we use the SIFTS annotation provided by EBI to filter by taxonomy. To learn more about how to [retrieve SIFTS annotation](
https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/demos/datasets/SiftsDataDemo.ipynb).

#### To filter by organism, we first retrieve the SIFTS taxonomy annotations on a PDB chain level
Here we are only interested in human proteins.

In [11]:
taxonomyQuery = "SELECT * FROM sifts.pdb_chain_taxonomy WHERE sifts.pdb_chain_taxonomy.scientific_name = 'Homo sapiens'"
taxonomy = pdbjMineDataset.get_dataset(taxonomyQuery).toPandas()
taxonomy.head()

,pdbid,chain,tax_id,scientific_name,structureChainId
0,10GS,A,9606,Homo sapiens,10GS.A
1,10GS,B,9606,Homo sapiens,10GS.B
2,11GS,A,9606,Homo sapiens,11GS.A
3,11GS,B,9606,Homo sapiens,11GS.B
4,121P,A,9606,Homo sapiens,121P.A


## Filter the PDB chains by joining with the taxonomy data

In [12]:
pd.set_option('display.max_columns', None)
pdb_filtered = pdb_map.merge(taxonomy, left_on=['structureId','chainId'], right_on=['pdbid','chain'], how='inner')
pdb_filtered = pdb_filtered.drop(['pdbid','chain'], axis=1)  # remove redundant columns
pdb_filtered['pdbPosition'] = pdb_filtered['pdbPosition'].astype('str') # must be string
pdb_filtered.head()

,alignmentId,bitscore,chainId,evalue,identity,identityPositive,midlineAlign,pdbAlign,pdbFrom,pdbId,pdbNo,pdbSeg,pdbTo,refGenome,residueMapping,segStart,seqAlign,seqFrom,seqId,seqTo,updateDate,variationId,structureId,pdbPosition,pdbAminoAcid,seqIdentity,tax_id,scientific_name,structureChainId
0,15184664,446.432,H,1.83875e-159,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2wju,2wju_H_1,1,221,hgvs-grch37,"[(G, 83, G, 83)]",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,2WJU,83,G,99.547511,9606,Homo sapiens,2WJU.H
1,15184672,446.432,H,1.83875e-159,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2vct,2vct_H_1,1,221,hgvs-grch37,"[(G, 83, G, 83)]",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,2VCT,83,G,99.547511,9606,Homo sapiens,2VCT.H
2,32437406,432.180,C,7.6822e-158,215.0,215.0,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,1,4acs,4acs_C_1,1,218,hgvs-grch37,"[(G, 83, G, 83)]",4,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,4,434735,221,2018-05-03,chr6:g.52619766C>T,4ACS,83,G,98.623853,9606,Homo sapiens,4ACS.C
3,17063512,993.416,A,0,465.0,466.0,ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...,ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...,1,2fo0,2fo0_A_1,1,466,hgvs-grch37,"[(Y, 272, Y, 253)]",65,ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...,46,488093,511,2017-10-25,chr9:g.133738358A>T,2FO0,272,Y,99.785408,9606,Homo sapiens,2FO0.A
4,17063513,961.444,A,0,450.0,451.0,DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...,DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...,1,1opl,1opl_A_1,1,451,hgvs-grch37,"[(Y, 272, Y, 253)]",81,DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...,62,488093,512,2017-10-25,chr9:g.133738358A>T,1OPL,272,Y,99.778271,9606,Homo sapiens,1OPL.A


In [13]:
chains = list(pdb_filtered.structureChainId)
print(chains)

['2WJU.H', '2VCT.H', '4ACS.C', '2FO0.A', '1OPL.A', '5MO4.A', '2F4J.A', '2G1T.C', '2G2F.A', '2GQG.A', '2HIW.B', '3QRI.A', '2V7A.A', '3UE4.A', '2G2H.B', '4ZOG.A', '2HZI.A', '3QRK.A', '4WA9.B', '4TWP.B', '1DXT.D', '1YZI.B', '1YHR.D', '1YHE.D', '1YH9.D', '1YDZ.D', '1Y8W.D', '1Y0D.D', '1Y0C.D', '1Y0A.D', '1Y09.D', '1XZV.D', '1XZU.D', '1XZ7.D', '1XZ5.D', '1XZ4.D', '1XZ2.D', '1XYE.D', '1XY0.D', '1XXT.D', '4XS0.B', '2W6V.D', '1VWT.D', '1QXD.D', '3KMF.G', '3KMF.C', '6FQF.D']


## Get PDB to UniProt Residue Mappings

Download PDB to UniProt mappings and filter out residues that were not observed in the 3D structure.

In [14]:
up = pdbToUniProt.get_cached_residue_mappings().filter("pdbResNum IS NOT NULL").filter("uniprotNum IS NOT NULL")
up_map = up.filter(up.structureChainId.isin(chains)).toPandas()
up_map['uniprotNum'] = up_map.uniprotNum.astype('int') 
                       
up_map.head()

,structureChainId,pdbResNum,pdbSeqNum,uniprotId,uniprotNum
0,1QXD.D,1,1,P68871,2
1,1QXD.D,2,2,P68871,3
2,1QXD.D,3,3,P68871,4
3,1QXD.D,4,4,P68871,5
4,1QXD.D,5,5,P68871,6


In [15]:
pdb_filtered = pdb_filtered.merge(up_map, left_on=['structureChainId','pdbPosition'], right_on=['structureChainId','pdbResNum'], how='inner')

In [16]:
pdb_filtered

,alignmentId,bitscore,chainId,evalue,identity,identityPositive,midlineAlign,pdbAlign,pdbFrom,pdbId,pdbNo,pdbSeg,pdbTo,refGenome,residueMapping,segStart,seqAlign,seqFrom,seqId,seqTo,updateDate,variationId,structureId,pdbPosition,pdbAminoAcid,seqIdentity,tax_id,scientific_name,structureChainId,pdbResNum,pdbSeqNum,uniprotId,uniprotNum
0,15184664,446.432,H,1.83875e-159,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2wju,2wju_H_1,1,221,hgvs-grch37,"[(G, 83, G, 83)]",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,2WJU,83,G,99.547511,9606,Homo sapiens,2WJU.H,83,83,P09210,83
1,15184672,446.432,H,1.83875e-159,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2vct,2vct_H_1,1,221,hgvs-grch37,"[(G, 83, G, 83)]",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2017-10-25,chr6:g.52619766C>T,2VCT,83,G,99.547511,9606,Homo sapiens,2VCT.H,83,83,P09210,83
2,32437406,432.180,C,7.6822e-158,215.0,215.0,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,1,4acs,4acs_C_1,1,218,hgvs-grch37,"[(G, 83, G, 83)]",4,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,4,434735,221,2018-05-03,chr6:g.52619766C>T,4ACS,83,G,98.623853,9606,Homo sapiens,4ACS.C,83,83,P09210,83
3,17063512,993.416,A,0,465.0,466.0,ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...,ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...,1,2fo0,2fo0_A_1,1,466,hgvs-grch37,"[(Y, 272, Y, 253)]",65,ARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVL...,46,488093,511,2017-10-25,chr9:g.133738358A>T,2FO0,272,Y,99.785408,9606,Homo sapiens,2FO0.A,272,230,P00519,253
4,17063513,961.444,A,0,450.0,451.0,DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...,DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...,1,1opl,1opl_A_1,1,451,hgvs-grch37,"[(Y, 272, Y, 253)]",81,DPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNG...,62,488093,512,2017-10-25,chr9:g.133738358A>T,1OPL,272,Y,99.778271,9606,Homo sapiens,1OPL.A,272,272,P00519,253
5,17063515,671.003,A,0,315.0,316.0,NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQG...,NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQG...,1,5mo4,5mo4_A_1,1,319,hgvs-grch37,"[(Y, 272, Y, 253)]",83,NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQG...,64,488093,382,2017-10-25,chr9:g.133738358A>T,5MO4,272,Y,98.746082,9606,Homo sapiens,5MO4.A,272,233,P00519,253
6,17063516,610.912,A,0,285.0,286.0,G+SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKT...,GMSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKT...,1,2f4j,2f4j_A_1,1,287,hgvs-grch37,"[(Y, 253, Y, 253)]",227,GVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKT...,227,488093,513,2017-10-25,chr9:g.133738358A>T,2F4J,253,Y,99.303136,9606,Homo sapiens,2F4J.A,253,27,P00519,253
7,17063529,595.504,C,0,276.0,276.0,YDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDT...,YDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDT...,1,2g1t,2g1t_C_1,1,276,hgvs-grch37,"[(Y, 253, Y, 253)]",232,YDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDT...,232,488093,507,2017-10-25,chr9:g.133738358A>T,2G1T,253,Y,100.000000,9606,Homo sapiens,2G1T.C,253,28,P00519,253
8,17063534,588.186,A,0,274.0,274.0,YDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDT...,YDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDT...,1,2g2f,2g2f_A_1,1,275,hgvs-grch37,"[(Y, 253, Y, 253)]",232,YDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDT...,232,488093,506,2017-10-25,chr9:g.133738358A>T,2G2F,253,Y,99.636364,9606,Homo sapiens,2G2F.A,253,28,P00519,253
9,17063535,587.030,A,0,272.0,272.0,SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLK...,SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLK...,7,2gqg,2gqg_A_1,1,278,hgvs-grch37,"[(Y, 253, Y, 253)]",223,SPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLK...,229,488093,500,2017-10-25,chr9:g.133738358A>T,2GQG,253,Y,100.000000,9606,Homo sapiens,2GQG.A,253,31,P00519,253


## Save mappings

In [17]:
pdb_filtered.to_csv(output_file_name, index=False)

## View mutations grouped by protein chain
Use the slider to view each protein chain.

In [18]:
chains = pdb_filtered.groupby('structureChainId')
print("Chains:", list(chains.groups.keys()))

Chains: ['1DXT.D', '1OPL.A', '1QXD.D', '1VWT.D', '1XXT.D', '1XY0.D', '1XYE.D', '1XZ2.D', '1XZ4.D', '1XZ5.D', '1XZ7.D', '1XZU.D', '1XZV.D', '1Y09.D', '1Y0A.D', '1Y0C.D', '1Y0D.D', '1Y8W.D', '1YDZ.D', '1YH9.D', '1YHE.D', '1YHR.D', '1YZI.B', '2F4J.A', '2FO0.A', '2G1T.C', '2G2F.A', '2G2H.B', '2GQG.A', '2HIW.B', '2HZI.A', '2V7A.A', '2VCT.H', '2W6V.D', '2WJU.H', '3KMF.C', '3KMF.G', '3QRI.A', '3QRK.A', '3UE4.A', '4ACS.C', '4TWP.B', '4WA9.B', '4XS0.B', '4ZOG.A', '5MO4.A', '6FQF.D']


In [19]:
def view_grouped_mutations(grouped_df, *args):
    chainIds = list(grouped_df.groups.keys())

    def view3d(show_bio_assembly=False, show_surface=False, show_labels=True, i=0):
        group = grouped_df.get_group(chainIds[i])
        
        pdb_id, chain_id = chainIds[i].split('.')
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly})

#       # polymer style
        viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})

#       # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
       
        # highlight chain of interest in blue
        viewer.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
        
        rows = group.shape[0]
        for j in range(0, rows):
            res_num = str(group.iloc[j]['pdbPosition'])
            mod_res = {'resi': res_num, 'chain': chain_id} 
            col = 'red'
            c_col = col + 'Carbon'
            viewer.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
            viewer.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}})          
            if show_labels:
                label = group.iloc[j]['variationId']
                viewer.addLabel(label, {'fontSize':7,'fontColor': 'black','backgroundColor':'ivory'}, {'resi': res_num, 'chain': chain_id})
        
        
        #print header
        print("PDB Id: " + pdb_id + " chain Id: " + chain_id)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])

        viewer.zoomTo({'chain': chain_id})
        
        if show_surface:
             viewer.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})
                
        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, i=s_widget)

In [20]:
view_grouped_mutations(chains);

PDB Id: 1DXT chain Id: D


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## View one mutation at a time
Use the slider to view each mutation. Surrounding residues within the `distance_cutoff` are rendered as orange sticks.

In [21]:
def view_single_mutation(df, distance_cutoff, *args):

    def view3d(show_bio_assembly=False, show_surface=False, show_labels=True, i=0):        
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly})

        # polymer style
        viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})

        # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
       
        # highlight chain of interest in green
        viewer.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue', 'opacity':0.5}})
        
        # 
        res_num = str(df.iloc[i]['pdbPosition'])
        label = df.iloc[i]['variationId']
        mod_res = {'resi': res_num, 'chain': chain_id} 
        col = 'red'
        c_col = col + 'Carbon'
        viewer.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
        viewer.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.8}})          
        
        if show_labels:
            viewer.addLabel(label, {'fontSize':12,'fontColor': 'black','backgroundColor':'ivory'}, {'resi': res_num, 'chain': chain_id})

        # select neigboring residues by distance
        surroundings = {'chain': chain_id, 'resi': res_num, 'byres': True, 'expand': distance_cutoff}
        
        # residues surrounding mutation positions
        viewer.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
        
        viewer.zoomTo(surroundings)
        
        if show_surface:
             viewer.addSurface(py3Dmol.SES, {'opacity':0.8,'color':'lightblue'}, {'chain': chain_id})
                
         #print header
        print("PDB Id:", pdb_id, "chain Id:" , chain_id, "residue:", res_num, "mutation:", label)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])
            
        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(df)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, i=s_widget)

In [22]:
view_single_mutation(pdb_filtered, distance_cutoff);

PDB Id: 2WJU chain Id: H residue: 83 mutation: chr6:g.52619766C>T


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [23]:
# Shutdown Spark
spark.stop()

## Now run the next step
Map mutations occuring at protein-polymer interfaces: [3-MapToPolymerInteractions.ipynb](3-MapToPolymerInteractions.ipynb)